In [1]:
start.time <- Sys.time()

geneExpressionFileName  <- 'gene_expression_n438x978.txt' 
myData <- read.table(geneExpressionFileName, sep="\t", header=T)
dimnames(myData)[[1]] <- myData[,1]
myData <- myData[,-1]
myData[1:5, 1:5]
dim(myData)
myData[1:10,1]
end.time <- Sys.time()
cat("took time: ",end.time - start.time)

,PSME1,ATF1,RHEB,FOXO3,RHOA
ACETAZOLAMIDE,-0.015159099,-0.031470528,-0.004733488,0.02591061,0.0056296773
IRBESARTAN,-0.026811981,0.012151979,-0.025550148,-0.02401181,-0.0106717396
IPRATROPIUM BROMIDE,0.001017958,-0.008650622,-0.018128698,-0.02079971,-0.0002722781
EFAVIRENZ,-0.004398264,0.055387992,0.004658520,0.01380732,-0.0340697348
THIAMINE,0.001838965,-0.018079188,-0.011855532,-0.03705033,-0.0133954959


[1] 438 978

[1] -0.0151590991 -0.0268119808  0.0010179582 -0.0043982642  0.0018389646
 [6] -0.0003519442  0.0225747085  0.0246360495  0.0169354604 -0.0030938307

took time:  1.010741

In [2]:
sideEffectFileName <- 'ADRs_HLGT_n438x232.txt'
sideEffectData <- read.table(sideEffectFileName, sep="\t", header=T)
dimnames(sideEffectData)[[1]] <- sideEffectData[,1]
sideEffectData <- sideEffectData[,-1]
sideEffectData[1:5, 1:5]
dim(sideEffectData)
names <- names(sideEffectData)
length(names)
names[1]
anyNA(sideEffectData)

,Abdominal.hernias.and.other.abdominal.wall.conditions,Abortions.and.stillbirth,Acid.base.disorders,Administration.site.reactions,Adrenal.gland.disorders
ACETAZOLAMIDE,0,0,1,1,0
IRBESARTAN,0,0,0,0,1
IPRATROPIUM BROMIDE,0,0,0,0,1
EFAVIRENZ,0,0,0,0,1
THIAMINE,0,0,0,0,0


[1] 438 232

[1] 232

[1] "Abdominal.hernias.and.other.abdominal.wall.conditions"

[1] FALSE

In [3]:
install.packages('e1071', dependencies=TRUE)
require('e1071')
install.packages('caret', dependencies = TRUE)
#require(caret)
#flds <- createFolds(y, k = 10, list = TRUE, returnTrain = FALSE)

Installing package into 'C:/Users/radeshpa/AppData/Local/Programs/Python/Python37/Lib/site-packages/rpy2/R/win-library/3.5'
(as 'lib' is unspecified)


package 'e1071' successfully unpacked and MD5 sums checked


Warning message:
"cannot remove prior installation of package 'e1071'"


The downloaded binary packages are in
	C:\Users\radeshpa\AppData\Local\Temp\RtmpmIUzj1\downloaded_packages


Loading required package: e1071
Installing package into 'C:/Users/radeshpa/AppData/Local/Programs/Python/Python37/Lib/site-packages/rpy2/R/win-library/3.5'
(as 'lib' is unspecified)


package 'caret' successfully unpacked and MD5 sums checked


Warning message:
"cannot remove prior installation of package 'caret'"


The downloaded binary packages are in
	C:\Users\radeshpa\AppData\Local\Temp\RtmpmIUzj1\downloaded_packages


In [4]:
# Install dependencies
library(caret)
library(e1071)
library(class)

ERROR: Error in library(caret): there is no package called 'caret'


In [ ]:
start.time <- Sys.time()
# Calculate t-score for all the genes
# All genes with  pValue less than threshold
#sideEffectData <- sideEffectData[1:438, 1:10]
sideEffectNames <- names(sideEffectData)
geneNames <- names(myData)
#sideEffectNames

## FOR EVERY SIDE EFFECT

experimentName <- "t_0_05_kNN_k = 10"
experimentResults <- data.frame(a = character(), b= numeric())
experimentResults <- rbind(experimentResults, data.frame(a = "side effect name", b = 0))
fileName <- paste(experimentName, '.csv')
experimentFile <- paste("C:\\Users\\radeshpa\\Desktop\\BioInformatics\\Assignment3\\Experiment_output", fileName)


for(i in 1:length(sideEffectNames)){
    sideEffectName <- sideEffectNames[i]
#res <- sapply(sideEffectNames, function(sideEffectName){
    cat("Working on the side effect: ", sideEffectName, "\n")
    # Y Values
    currentSideEffect <- sideEffectData[,sideEffectName]
    presentIndices <- which(currentSideEffect == 1)
#    length(presentIndices)
    absentIndices <- which(currentSideEffect == 0)
#    length(absentIndices)

    
    # Create frame only using subset data.
    subSetData <- myData[, featuresOfInterest]
#    subSetData <- data.frame (myData$UBE2C, myData$ATF1, myData$RHOA)   
    
    cat("gene data: ", dim(subSetData), " Count of side effect: ", length(currentSideEffect), "\n")
   
    outerCount <- 3
    accuracyOverFoldsTotal <- 0
    
    
    for(j in 1:outerCount){
        
        
################################################  DATA  ###########################
        
        foldCount <- 10
        folds <- createFolds(currentSideEffect, foldCount, returnTrain = FALSE)
        # Get the values for every list (indices)

        foldIndices <- list()
        
        for(k in 1:length(folds)){
            foldIndices[[length(foldIndices)+1]] <- flds[[k]]
        }

        accuracyOverFolds <- 0
        accuracyOverFoldsSum <- 0
       
        
        for(k in 1:length(folds)){
            # Data of this fold.
            cat("Running fold number: ", k, " Count number: ", j, "\n")

            # We need to create a vector for indexing
            testingDataX <- myData[foldIndices[[k]],]

#            cat("Dimension of testying data x = ", dim(testingDataX), "\n")

            testingDataY <- sideEffectData[foldIndices[[k]],sideEffectName]

#            cat("Length of testing data = ", length(testingDataY), "\n")


            remainingIndices <- (Reduce(c,foldIndices[-k]))

            trainingDataX <- myData[c(remainingIndices), ]

            trainingDataY <- sideEffectData[c(remainingIndices), sideEffectName]


###################################################### FEATURE SELECTION : None #####################################################

            # featuresOfInterest <- names(trainingDataX)
            
##################################################### FEATURE SELECTION : t- statistic ###########################################################            

            allpValues <- sapply(geneNames, function(geneName){
                geneData <- trainingDataX[,geneName]
                sampleOne <-  geneData[presentIndices]
                sampleOneLength <- length(sampleOne)
                sampleTwo <-  geneData[absentIndices]
                sampleTwoLength <- length(sampleTwo)
        #        cat("sample one size = ", length(sampleOne), " and sample two size = ", length(sampleTwo), "\n")
                if(sampleOneLength == 0 | sampleTwoLength == 0 | (sampleOneLength == 1 && sampleTwoLength != 1) | (sampleOneLength != 1 && sampleTwoLength == 1)){
                    # this is a great feature!
                    pValue  <- 0 # we want this feature.
                }
                else{
        #           cat("sample one size = ", length(sampleOne), " and sample two size = ", length(sampleTwo), "\n")

                    pValue <- t.test(sampleOne, sampleTwo)$p.value
        #            cat(" gene: ", geneName, " side effect: ", sideEffectName, "  p Value : ", pValue, "\n")
                }
                pValue            
            })


            threshold <- 0.05
            sigpIndices <- which(allpValues < threshold)

            sigpValues <- allpValues[sigpIndices]
            featuresOfInterest <- names(trainingDataX)[sigpIndices]

            refinedTrainingDataX <- trainingDataX[, featuresOfInterest]

            
########################################## KNN #######################################################################################

            # Now filter the test set also!
            refinedTestingDataX <- testingDataX[,featuresOfInterest]
            predictionResults <- knn(refinedTrainingDataX, refinedTestingDataX, factor(trainingDataY), k = 10)



            accuracy <- (sum(predictionResults==testingDataY) + sum(is.na(predictionResults) & is.na(testingDataY))) / length(predictionResults)
#            cat("Accuracy for fold:  ", i, " is ", accuracy, "\n")
            accuracyOverFoldsSum <- accuracyOverFoldsSum + accuracy
            
    #######################################################################################################################        

        }
        
        accuracyOverFolds <- (accuracyOverFoldsSum)/length(folds)     
        accuracyOverFoldsTotal <- accuracyOverFoldsTotal + accuracyOverFolds
    }
    
    finalAccuracy  <- accuracyOverFoldsTotal/outerCount
    cat("Final Accuracy is: ", finalAccuracy, "\n")
    
    write.table(data.frame(a = sideEffectName, b = finalAccuracy), experimentFile, append = TRUE, sep = "\t", row.names = FALSE, col.names = FALSE)    
}

end.time <- Sys.time()
cat("took time: ",end.time - start.time)